#Ingestion del archivo movie_cast.json


### Paso 1 - Leer el archivo JSON usando "DataframeReader"

In [0]:
dbutils.widgets.text("p_environment", "")
v_enviroment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run 
"../includes/common_functions"

In [0]:
%run
"../includes/configuration"

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, TimestampType

In [0]:
language_rol_schema = StructType(fields=[
    StructField("roleId", IntegerType(), False),
    StructField("languageRole", StringType(), True)
])

In [0]:
language_rol_df = spark.read \
    .schema(language_rol_schema) \
    .option("multiline", True) \
    .json(f"{bronze_folder_path}/{v_file_date}/language_role.json")


## Parte 2 : Renombrar las columnas y añadir nuevas columnas
##### 1. "personId" renombrar a "person_id"
##### 2. Agregar las columnas "Ingestion_date" y "environment"
##### 3. Agregar la columna "name" a partir de la concatenación de "forename" y "surname"

In [0]:
from pyspark.sql.functions import col,concat,current_timestamp,lit



In [0]:
language_rol_final_df = language_rol_df \
                                .withColumnsRenamed({"roleId": "role_id",
                                                     "languageRole": "language_role"}) \
                                .withColumn("ingestion_date", current_timestamp()) \
                                .withColumn("enviroment", lit("Production")) \
                                .withColumn("file_date", lit(v_file_date))





## Parte 3. guardar en formato parquet en el datalake

In [0]:
language_rol_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.language_role")
#merge_delta_data(language_rol_final_df,"/mnt/moviehistory7809/silver","movie_silver", "language_role", "file_date", "role_id")

In [0]:
%sql
select * from movie_silver.language_role


In [0]:
dbutils.notebook.exit("Success No8")